In [ ]:
import os
import zipfile
import hashlib

def zip_files(folder_path, zip_file_name):
    """Zip all files in a folder."""
    # Create a ZipFile object
    with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        file_count = 0  # To count the number of files added to the zip
        # Walk through the folder
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                # Add file to zip archive
                zipf.write(file_path, os.path.relpath(file_path, folder_path))
                file_count += 1  # Increment file count for each file added
    print(f"Files in '{folder_path}' have been zipped to '{zip_file_name}'.")
    return file_count  # Return the total number of files

def generate_md5(file_path):
    """Generate MD5 checksum for a file."""
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

def create_md5_info_file(zip_file_path, info_file_path, date, file_count):
    """Create an MD5 .info file for the zip file."""
    md5_checksum = generate_md5(zip_file_path)
    with open(info_file_path, 'w') as info_file:
        info_file.write(f"filename: '{os.path.basename(zip_file_path)}'\n")
        info_file.write(f"md5: '{md5_checksum}'\n")
        info_file.write(f"date: '{date}'\n")  # Assuming date is passed from your code
        info_file.write(f"file_count: '{file_count}'\n")
    print(f"MD5 checksum and details written to '{info_file_path}'.")

def main():
    # Define paths
    folder_to_zip = 'your_folder_path'  # Replace with the folder containing your files
    zip_file_name = 'output_files.zip'
    md5_info_file_name = 'output_files.info'
    date = '2024-09-09'  # Example date; replace this with your date variable

    # Zip the files in the specified folder
    file_count = zip_files(folder_to_zip, zip_file_name)

    # Create an MD5 .info file for the generated zip file
    create_md5_info_file(zip_file_name, md5_info_file_name, date, file_count)

if __name__ == "__main__":
    main()